## rfr

### Libraries and data

In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics.scorer import make_scorer
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
import pickle
from sklearn.externals import joblib 

print('Readyyy')

Readyyy


/home/boi/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
train_df = pd.read_excel('train_df.xlsx')
test_df = pd.read_excel('test_df.xlsx')

X = train_df.drop(columns = ['Price'])
y = train_df.Price

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 7)

### Randomforest regressor

In [74]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 500,random_state = 7)
rfr.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=7, verbose=0,
                      warm_start=False)

In [75]:
rfr.score(X_val, y_val)

0.8953788774696806

In [94]:
rfr.get_params

<bound method BaseEstimator.get_params of RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=7, verbose=0,
                      warm_start=False)>

In [76]:
def rmsle(Y,YH):
    s = 0
    for y,yh in zip(Y,YH):
        s += (np.log(y)-np.log(yh)) ** 2
    return (s / (Y.shape[0])) ** 0.5

In [77]:
y_pred_val = rfr.predict(X_val)

score = rmsle(y_val, y_pred_val)
print(1 - score)

0.867976843198199


In [78]:
scores = cross_val_score(rfr, X, y, cv = 5, scoring = make_scorer(rmsle, greater_is_better = True))

scores = [1 - s for s in scores]
scores = np.array(scores)
scores

array([0.86840251, 0.87428925, 0.87149643, 0.86848298, 0.86298578])

In [79]:
scores.mean()

0.8691313898142201

5CV score = 86.9131% without performing hyperparameter tuning

In [103]:
# Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 400, stop = 1600, num = 5)]
# Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(4, 8, num = 6)][1:]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [3, 5, 9]
# Method of selecting samples for training each tree
# bootstrap = [True, False]
# Create the random grid
random_grid = { 'n_estimators' : [1000],
               'max_depth': max_depth,
               'min_samples_split': [2],
               'min_samples_leaf': [3]}

pprint(random_grid)

{'max_depth': [4, 5, 6, 7, 8, None],
 'min_samples_leaf': [3],
 'min_samples_split': [2],
 'n_estimators': [1000]}


In [104]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose = 2, random_state = 7, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.3min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_state=N

In [109]:
rf_random.best_params_

AttributeError: 'RandomForestRegressor' object has no attribute 'best_params_'

```{'n_estimators': 700,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 8,
 'bootstrap': True}```  gave a cv score of 82
 
 ```{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 3,
 'max_depth': 8}``` 83

In [105]:
rf_random = RandomForestRegressor(n_estimators = 1000, min_samples_split = 2, min_samples_leaf = 3, max_depth = None ,random_state = 7)

In [106]:
rf_random.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=7, verbose=0,
                      warm_start=False)

In [107]:
rf_random.score(X_val, y_val)

0.8830244619113752

In [108]:
scores = cross_val_score(rf_random, X, y, cv = 5, scoring = make_scorer(rmsle, greater_is_better = True))

scores = [1 - s for s in scores]
scores = np.array(scores)
scores, scores.mean()

(array([0.86289517, 0.86862322, 0.86534549, 0.86412913, 0.85699769]),
 0.8635981415427881)

In [112]:
joblib.dump(rf_random, 'rf_regressor.pkl')

['rf_regressor.pkl']